In [25]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from helpers import *
from implementations import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Load the training data into feature matrix, class labels, and event ids:

In [15]:
y_train, x_train, _ = load_csv_data('data/train.csv')

#### Sanitizing the missing values and standardize the features

In [17]:
# JULES FLO: si on sanitize après il y avait des nan jsp pourquoi, amdahl
x_train_san = sanitize(x_train)
y_train = y_train.reshape((y_train.shape[0], 1))

#### Discrete feature

In [19]:
feature_22_train = x_train_san[:, 22]
categories_22 = np.unique(feature_22_train)
x_train_san_minus_22 = x_train_san[:, np.array(range(x.shape[1])) != 22]
x_train_sep = []
y_train_sep = []

for i in categories_22: # Assuming same set of category in the training and testing
    x_train_sep.append(x_train_san_minus_22[feature_22_train == i, :])
    y_train_sep.append(y_train[feature_22_train == i, :])

In [20]:
# Rebuilding final y for submission, in progress
N = x_test.shape[0]
predictions = np.zeros(N)
for i in categories_22:
    ind = np.arange(N)[np.isclose(i, features_22_test, atol=1e-04)]
    predictions[ind] = 

SyntaxError: invalid syntax (<ipython-input-20-0c184010f8fd>, line 5)

In [27]:
accuracies = []
weights = []
for i, x_chunk in enumerate(x_train_sep):
    w_init = np.random.rand(x_chunk.shape[1], 1)
    w, loss = least_squares_GD(y_train_sep[i], x_chunk, w_init, 1000, 0.01, verbose=True) 
    y_pred = predict_labels(w, x_chunk)
    accuracies.append(compute_accuracy_linear_reg(y_train_sep[i], y_pred))
    weights.append(w)
    
accuracies

Iteration : 0 with loss 6.450879851498423 and gradient norm: 5.2949655558829996
Iteration : 250 with loss 0.33888191557092795 and gradient norm: 0.23865532916917823
Iteration : 500 with loss 0.28938969362720307 and gradient norm: 0.06947344049309992
Iteration : 750 with loss 0.28333925216863215 and gradient norm: 0.03555978862893712
Iteration : 0 with loss 7.452108683713229 and gradient norm: 5.818346417752108
Iteration : 250 with loss 0.44345963076205164 and gradient norm: 0.21575812346784867
Iteration : 500 with loss 0.4012819407366933 and gradient norm: 0.0727687613696616
Iteration : 750 with loss 0.3930971455255257 and gradient norm: 0.046254790474539606
Iteration : 0 with loss 13.67910152265491 and gradient norm: 12.796020211909392
Iteration : 250 with loss 0.5159878990640682 and gradient norm: 0.3421576311697902
Iteration : 500 with loss 0.391543583051741 and gradient norm: 0.1334200865209421
Iteration : 750 with loss 0.3675070379449254 and gradient norm: 0.06923134444376204
Iter

[0.8136979171879535,
 0.7054317548746518,
 0.7315746640465273,
 0.7268092402093485]

In [ ]:
# Old but kept because can be useful as basis
N = sanitized_x.shape[0]
inds = np.random.choice(range(N), 100000, replace=False)
sanitized_x_sub = sanitized_x[inds, :]
sanitized_x_sub = sanitized_x_sub[:, [0,1,2,3,7,8,9,10,11,13,14,17,19,21,22,29]]
sanitized_y_sub = sanitized_y[inds]
x_train, x_test, y_train, y_test = split_data(sanitized_x, sanitized_y, 0.8)
number_of_w = 1

for degree in range(1, 5)
    tx_train = build_poly(x_train, degree)
    tx_test = build_poly(x_test, degree)
    for gamma in np.array([0.01, 0.05, 0.1, 0.5]):
        print("Running logistic regression with polynomial feature expansion with degree", degree, 
                 "and gamma", gamma,", with", number_of_w, "different initial weights.")
        for i in  range(number_of_w):
            w_init = np.random.rand(tx_train.shape[1], 1)
            w, loss = logistic_regression(y_train, tx_train, w_init, 1000, gamma, verbose=True)
            print(i,":")
            print("Accuracy on training set", compute_accuracy(y_train, tx_train, w))
            print("Accuracy on test set:, ", compute_accuracy(y_test, tx_test, w))

## Generate predictions and save ouput in csv format for submission:


In [ ]:
_, x_test, ids_test = load_csv_data('data/test.csv')

In [ ]:
x_test_san  = sanitize(x_test)

feature_22_test = x_test_san[:, 22]
x_test_san_minus_22 = x_test_san[:, np.array(range(x.shape[1])) != 22]
x_test_sep = []

for i in categories_22:
    x_test_sep.append(x_test_san_minus_22[feature_22_test == i, :])

In [ ]:
y_pred = predict_labels(weights, x_test)
# Rebuild original form for submission

In [ ]:
OUTPUT_PATH = 'predictions.csv'
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)